In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [3]:
store_files = [
    "synthetic_data/RCT_0_5.h5",
    "synthetic_data/RCT_0_05.h5",
    "synthetic_data/e_X.h5",
    "synthetic_data/e_X_U.h5",
    "synthetic_data/e_X_no_overlap.h5",
    "synthetic_data/e_X_info_censor.h5",
    "synthetic_data/e_X_U_info_censor.h5",
    "synthetic_data/e_X_no_overlap_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            continue
    experiment_setups[base_name] = scenario_dict

In [4]:
experiment_setups['RCT_0_5']['scenario_1']['dataset']

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0.281175,1,0,0.484593,0.998236,0.668208,0.070638,0.960140,0.497815,0.206792,0.281175,0.061038,0.281175,1.365563
49996,49996,0.029867,1,1,0.036391,0.268106,0.043117,0.426886,0.342038,0.812595,0.437775,0.163239,0.029867,0.029867,0.658388
49997,49997,0.077500,1,1,0.061915,0.411210,0.426204,0.414266,0.601355,0.116056,0.416950,0.200592,0.077500,0.077500,1.118571
49998,49998,0.423983,1,0,0.178390,0.656522,0.817355,0.347013,0.060741,0.201218,0.935754,0.423983,0.205703,0.423983,0.939400


In [5]:
experiment_repeat_setups = pd.read_csv("synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# EXPERIMENT CONSTANTS

In [6]:
NUM_REPEATS_TO_INCLUDE = 10  # max 10
NUM_TRAINING_DATA_POINTS = 10000 # max 45000
TEST_SIZE = 5000

In [7]:
output_pickle_path = f"results/csf_num_repeats_{NUM_REPEATS_TO_INCLUDE}_train_size_{NUM_TRAINING_DATA_POINTS}.pkl"

# Run Experiments

In [8]:
from models_causal_survival.causal_survival_forest import CausalSurvivalForestGRF
from sklearn.metrics import mean_squared_error
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [9]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list):
    split_results = {}

    for rand_idx in random_idx_col_list:
        random_idx = experiment_repeat_setups[rand_idx].values
        test_ids = random_idx[-TEST_SIZE:]
        train_ids = random_idx[:min(NUM_TRAINING_DATA_POINTS, len(random_idx) - TEST_SIZE)]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        cate_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[rand_idx] = (X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true)

    return split_results

In [ ]:
random_idx_col_list = experiment_repeat_setups.columns.to_list()[:NUM_REPEATS_TO_INCLUDE]
failure_times_grid_size = 500

print("Output results path:", output_pickle_path)

results_dict = {}

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):
     
    results_dict[setup_name] = {}

    for scenario_key in tqdm(setup_dict, desc=f"{setup_name} Scenarios", leave=False):
        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list)

        # Initialize results dictionary for this setup and scenario
        results_dict[setup_name][scenario_key] = {}

        start_time = time.time()

        for rand_idx in random_idx_col_list:
            X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true = split_dict[rand_idx]

            # Store placeholder for later population
            results_dict[setup_name][scenario_key][rand_idx] = {}

            # Train the model
            csf = CausalSurvivalForestGRF(failure_times_grid_size=failure_times_grid_size)
            csf.fit(X_train, W_train, Y_train)

            # Predict CATE
            # cate_test_pred = csf.predict_cate(X_test, W_test)
            mse_test, cate_test_pred, ate_test_pred = csf.evaluate(X_test, cate_test_true, W_test)

            # Save results
            results_dict[setup_name][scenario_key][rand_idx] = {
                "cate_true": cate_test_true,
                "cate_pred": cate_test_pred,
                "ate_true": cate_test_true.mean(),
                "ate_pred": ate_test_pred,
                "cate_mse": mse_test,
                "ate_bias": ate_test_pred - cate_test_true.mean(),
            }

        end_time = time.time()

        # Save results to the setup dictionary
        results_dict[setup_name][scenario_key]["average"] = {
            "mean_cate_mse": np.mean([results_dict[setup_name][scenario_key][rand_idx]["cate_mse"]
                                      for rand_idx in random_idx_col_list]),
            "std_cate_mse": np.std([results_dict[setup_name][scenario_key][rand_idx]["cate_mse"]
                                    for rand_idx in random_idx_col_list]),
            "mean_ate_pred": np.mean([results_dict[setup_name][scenario_key][rand_idx]["ate_pred"]
                                      for rand_idx in random_idx_col_list]),
            "std_ate_pred": np.std([results_dict[setup_name][scenario_key][rand_idx]["ate_pred"]
                                    for rand_idx in random_idx_col_list]),
            "mean_ate_true": np.mean([results_dict[setup_name][scenario_key][rand_idx]["ate_true"]
                                      for rand_idx in random_idx_col_list]),
            "std_ate_true": np.std([results_dict[setup_name][scenario_key][rand_idx]["ate_true"]
                                    for rand_idx in random_idx_col_list]),
            "mean_ate_bias": np.mean([results_dict[setup_name][scenario_key][rand_idx]["ate_bias"]
                                      for rand_idx in random_idx_col_list]),
            "std_ate_bias": np.std([results_dict[setup_name][scenario_key][rand_idx]["ate_bias"]
                                    for rand_idx in random_idx_col_list]),
            "runtime": (end_time - start_time) / len(random_idx_col_list)
        }

        # Save progress to disk
        with open(output_pickle_path, "wb") as f:
            pickle.dump(results_dict, f)

        # break
    # break
            


Output results path: results/csf_num_repeats_10_train_size_10000.pkl


Experiment Setups:   0%|          | 0/8 [00:00<?, ?it/s]

R[write to console]: Warning in (function (X, Y, W, D, W.hat = NULL, target = c("RMST", "survival.probability"),  :
R[write to console]: 
 
R[write to console]:  Estimated censoring probabilities go as low as: 0.02319 - an identifying assumption is that there exists a fixed positive constant M such that the probability of observing an event past the maximum follow-up time  is at least M (i.e. P(T > horizon | X) > M). This warning appears when M is less than 0.05, at which point causal survival forest can not be expected to deliver reliable estimates.

R[write to console]: Warning in (function (X, Y, W, D, W.hat = NULL, target = c("RMST", "survival.probability"),  :
R[write to console]: 
 
R[write to console]:  Estimated censoring probabilities go as low as: 0.02464 - an identifying assumption is that there exists a fixed positive constant M such that the probability of observing an event past the maximum follow-up time  is at least M (i.e. P(T > horizon | X) > M). This warning appears 

In [ ]:
def summarize_experiment_results(results_dict):
    records = []

    for setup_name, setup_dict in results_dict.items():
        for scenario_key in setup_dict:
            avg_result = setup_dict[scenario_key].get("average", {})
            mean_mse = avg_result.get("mean_cate_mse", np.nan)
            std_mse = avg_result.get("std_cate_mse", np.nan)
            mean_ate_pred = avg_result.get("mean_ate_pred", np.nan)
            std_ate_pred = avg_result.get("std_ate_pred", np.nan)
            mean_ate_true = avg_result.get("mean_ate_true", np.nan)
            std_ate_true = avg_result.get("std_ate_true", np.nan)
            mean_ate_bias = avg_result.get("mean_ate_bias", np.nan)
            std_ate_bias = avg_result.get("std_ate_bias", np.nan)
            runtime = avg_result.get("runtime", np.nan)

            records.append({
                "setup_name": setup_name,
                "scenario_key": scenario_key,
                "CATE_MSE": f"{mean_mse:.3f} ± {std_mse:.3f}" if not pd.isna(mean_mse) else np.nan,
                "ATE_pred": f"{mean_ate_pred:.3f} ± {std_ate_pred:.3f}" if not pd.isna(mean_ate_pred) else np.nan,
                "ATE_true": f"{mean_ate_true:.3f} ± {std_ate_true:.3f}" if not pd.isna(mean_ate_true) else np.nan,
                "ATE_bias": f"{mean_ate_bias:.3f} ± {std_ate_bias:.3f}" if not pd.isna(mean_ate_bias) else np.nan,
                "runtime [s]": round(runtime) if not pd.isna(runtime) else np.nan
            })

    df = pd.DataFrame.from_records(records)
    return df

In [ ]:
summary_df = summarize_experiment_results(results_dict)
summary_df

,setup_name,scenario_key,CATE_MSE,ATE_pred,ATE_true,ATE_bias,runtime [s]
0,RCT_0_5,scenario_1,0.683 ± 0.068,0.128 ± 0.033,0.126 ± 0.011,0.002 ± 0.031,4
1,RCT_0_5,scenario_2,14.044 ± 1.845,0.040 ± 0.144,0.165 ± 0.057,-0.125 ± 0.147,3
2,RCT_0_5,scenario_5,14.666 ± 0.393,0.716 ± 0.062,0.751 ± 0.060,-0.036 ± 0.090,3
3,RCT_0_5,scenario_8,16.554 ± 0.288,0.569 ± 0.151,0.775 ± 0.042,-0.207 ± 0.152,3
4,RCT_0_5,scenario_9,42.896 ± 3.947,0.063 ± 0.111,0.729 ± 0.086,-0.666 ± 0.153,4
5,RCT_0_05,scenario_1,0.683 ± 0.066,0.142 ± 0.050,0.126 ± 0.011,0.015 ± 0.051,3
6,RCT_0_05,scenario_2,15.248 ± 4.076,0.018 ± 0.925,0.165 ± 0.057,-0.146 ± 0.927,3
7,RCT_0_05,scenario_5,14.744 ± 0.366,0.621 ± 0.227,0.751 ± 0.060,-0.130 ± 0.214,2
8,RCT_0_05,scenario_8,17.041 ± 0.728,0.261 ± 0.321,0.775 ± 0.042,-0.514 ± 0.299,2
9,RCT_0_05,scenario_9,42.662 ± 4.018,0.286 ± 0.234,0.729 ± 0.086,-0.443 ± 0.235,3
